# Setup

In [1]:
from piper import piper 
from piper.verbs import *
from piper.pandas import *
from pathlib import Path

piper version 0.0.9, last run: Tuesday, 09 March 2021 20:33:28


# Helper functions

## clean_postcodes

In [2]:
def clean_postcodes(filename):
    
    df = pd.read_csv(filename, header=None, usecols=[0], names=['post_code'])
    
    df[['p1', 'p2']] = df.post_code.str.extract(r'(.*)(\d\w{,2})$', expand=True)
    df['post_code'] = df.p1 + ' ' + df.p2
    df['postcode'] = df.p1.str.extract('([a-zA-Z]+)')
    
    df.drop(columns=['p1', 'p2'], inplace=True)
    
    return df

# Postcodes

## Consolidate raw multiple CSV files

In [5]:
directory = Path('inputs/fake_data/supporting workfiles/CSV')
list_of_csv_files = list_files(directory, glob_pattern='*.csv', as_posix=True)

dataframes = [clean_postcodes(f) for f in list_of_csv_files]

df = pd.concat(dataframes).dropna()

In [6]:
head(df)

1708495 rows, 2 columns


,post_code,postcode
0,WR1 1AA,WR
1,WR1 1AD,WR
2,WR1 1AE,WR
3,WR1 1AF,WR


## Read top level postcodes

In [7]:
directory = Path('inputs/fake_data/')

In [8]:
filename = directory / 'UK_Toplevel_postcode_areas.csv'
subset_cols = ['postcode', 'area_covered', 'center_latitude', 'center_longitude']

%piper areas <- read_csv(filename, sep='\t', info=False) >> select(subset_cols)

inputs/fake_data/UK_Toplevel_postcode_areas.csv
121 rows, 9 columns


In [9]:
head(areas)

121 rows, 4 columns


,postcode,area_covered,center_latitude,center_longitude
0,AB,Aberdeen,57.3010,-2.307900
1,AL,St Albans,51.7755,-0.283982
2,B,Birmingham,52.4652,-1.888850
3,BA,Bath,51.2295,-2.417340


## Merge with detail and write TSV

In [10]:
%%piper 

df 
>> inner_join(areas, on='postcode')
>> order_by(['post_code', 'center_latitude', 'center_longitude'])
>> to_csv(directory / 'UK postcodes.tsv', sep='\t')

## Check:: Count postcodes by area_covered

In [11]:
%%piper
read_csv(directory / 'UK postcodes.tsv', sep='\t')
>> count('area_covered')
>> adorn()

inputs/fake_data/UK postcodes.tsv
1708495 rows, 5 columns


,n,%,cum %
Birmingham,41444,2.43,2.43
Sheffield,33482,1.96,4.39
Newcastle upon Tyne,33185,1.94,6.33
Glasgow,31815,1.86,8.19
Manchester,31720,1.86,10.05
...,...,...,...
Llandrindod Wells,2202,0.13,99.80
Kirkwall,1836,0.11,99.91
Outer Hebrides,961,0.06,99.96
Lerwick,650,0.04,100.00
